#  <span style="color:orange">Regression Tutorial (REG102) - Level Intermediate</span>

**Created using:PyCaret 2.2** <br /> **Date Updated:November 25, 2020**

# 1.0 Tutorial Objective 回帰チュートリアル

**(REG102)** - Level Intermediate へようこそ。このチュートリアルは、__[回帰チュートリアル(REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__を完了していることを前提としています。まだPyCaretを使ったことがなく、このチュートリアルが初めての場合は、ビギナーチュートリアルに戻ってPyCaretでの作業の基本を理解することを強くお勧めします。

このチュートリアルでは、`pycaret.regression`モジュールを使って、以下のことを学びます。

* **Normalization:** データセットを正規化し、スケーリングする方法 
* **Transformation:** データを線形かつ近似的に正規化するための変換を適用する方法 
* **Target Transformation:** ターゲット変数に変換を適用する方法 
* **Combine Rare Levels:** カテゴリカル変数で稀少レベルを組み合わせる方法
* **Bin Numeric Variables:** 数値変数をビンに入れ、「sturges」ルールを使用して数値特徴をカテゴリーに変換する方法 
* **Model Ensembling and Stacking:** Bagging、Boosting、Voting、Generalized Stackingなどのアンサンブル技術を使用して、モデルのパフォーマンスを向上させる方法。
* **Experiment Logging:** MLFlowバックエンドを使ったPyCaretでの実験ログの取り方

読了時間：約60分


# 1.1 PyCaretのインストール まだPyCaretをインストールしていない場合。

[初心者向けチュートリアル](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__のリンクを参照して、pycaretのインストール方法を確認してください。

# 1.2 前提条件 - Python 3.6以上 - PyCaret 2.0以上 - 

pycaretのリポジトリからデータを読み込むためのインターネット接続環境 - Regression Tutorial (REG101)の修了 - Level Beginner

  `from pycaret.utils import enable_colab`  `enable_colab()`

# 1.4 See also: - __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ - __[Regression Tutorial (REG103) - Level Expert](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Expert%20-%20REG103.ipynb)__

# 2.0 

このチュートリアルで扱う技術の概要 セクション1で述べた技術を実際に実行する前に、これらの技術が何であるか、いつ使用するかを理解することが重要です。多くの場合、これらのテクニックは、線形アルゴリズムやパラメトリック・アルゴリズムに役立ちますが、ツリーベースのモデルでもパフォーマンスが向上することは驚くことではありません。以下の説明は簡単なものですので、これらのテクニックをより深く理解するためには、追加の読み物を読むことをお勧めします。

- **正規化：** 正規化/スケーリング（標準化と同じ意味で使われることが多い）は、機械学習に役立つ特性を提供する方法で、数値変数の実際の値を変換するために使用されます。線形回帰、サポートベクターマシン、K Nearest Neighborsなどの多くのアルゴリズムは、すべての特徴がゼロを中心とし、同じレベルのオーダーの分散を持つことを前提としています。データセット内の特定の特徴が、他の特徴よりも大きいオーダーの分散を持っている場合、モデルはすべての特徴を正しく理解していない可能性があり、パフォーマンスが低下する可能性があります。_[続きを読む](https://sebastianraschka.com/Articles/2014_about_feature_scaling.html#z-score-standardization-or-min-max-scaling)__   
- **変換：** 正規化が分散の大きさの影響を取り除くためにデータの範囲を変換するのに対し、変換は、変換されたデータが正規または近似正規分布で表現できるように分布の形を変えるという、より根本的な手法です。一般に、正規性やガウス分布を仮定したアルゴリズムを使用する場合は、データを変換する必要があります。このようなモデルの例としては、線形回帰、ラッソ回帰、リッジ回帰などがあります。__[Read more](https://en.wikipedia.org/wiki/Power_transform)__  
- **Target Transformation:** これは上で説明した`transformation`テクニックと似ていますが、例外的にこれはターゲット変数にのみ適用されます。
- **Combine Rare Levels:** カテゴリー特徴には、頻度分布では重要でないレベルがあることがあります。このような場合、学習用のサンプルサイズが限られているため、データセットにノイズをもたらす可能性があります。稀なレベルを扱う1つの方法は、それらを新しいクラスに結合することです。 
- **数値変数のビン化：** ビン化または離散化は、数値変数をカテゴリー特徴に変換するプロセスです。例えば、この実験では「カラットの重さ」を例に挙げています。これは数値の連続的な分布で、間隔に離散化することができます。ビン化は、データのノイズや非線形性を低減することで、予測モデルの精度を向上させることができます。PyCaretでは、Sturgesルールを用いて、ビンの数とサイズを自動的に決定します。  _[続きを読む](https://www.vosesoftware.com/riskwiki/Sturgesrule.php)__ 
- **モデルのアンサンブルとスタッキング:**  アンサンブルモデリングとは、ある結果を予測するために複数の多様なモデルを作成するプロセスです。これは、多くの異なるモデリングアルゴリズムを使用するか、またはトレーニングデータセットの異なるサンプルを使用することによって達成されます。そして、アンサンブルモデルは、各ベースモデルの予測を集約し、未見のデータに対する最終的な予測を1つにまとめます。アンサンブルモデルを使用する動機は、予測の汎化誤差を低減することにあります。ベースモデルが多様で独立している限り、アンサンブルアプローチを使用すると、モデルの予測誤差は減少します。アンサンブル学習で最も一般的な手法は`Bagging`と`Boosting`です。スタッキングもアンサンブル学習の一種であり、複数のモデルからの予測値を、最終的な結果を予測するメタモデルの入力特徴として使用します。__[続きを読む](https://blog.statsbot.co/ensemble-learning-d1dcd548e936)__
- **Tuning Hyperparameters of Ensemblers:** 単一の機械学習モデルのハイパーパラメータのチューニングと同様に、アンサンブルモデルのハイパーパラメータのチューニング方法も学びます。

# 3.0 Dataset for the Tutorial

このチュートリアルでは、__[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__で使用したのと同じデータセットを使用します。

# Dataset 謝辞: このケースは、Phillip E. Pfeifer, Alumni Research Professor of Business Administrationの監督のもと、Greg Mills (MBA '07)によって作成されました。Copyright (c) 2007 by the University of Virginia Darden School Foundation, Charlottesville, VA.すべての権利は留保されています。

オリジナルのデータセットと説明は、__[ここにあります。](https://github.com/DardenDSC/sarah-gets-a-diamond)__。

# 4.0 Getting the Data

オリジナルのソース__[found here](https://github.com/DardenDSC/sarah-gets-a-diamond)__からデータをダウンロードして、pandas read_csv関数を使って読み込むか、PyCaretのデータリポジトリを使ってget_data関数を使ってデータを読み込むことができます（これにはインターネット接続が必要です）。

In [1]:
from pycaret.datasets import get_data
dataset = get_data('diamond', profile=True)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

profile "パラメータが "True "の場合、探索的データ解析のためのデータプロファイルが表示されることに注意してください。この分析に基づいて、上記のセクション2で説明したようないくつかの前処理ステップがこの実験で実行されます。このプロファイルが、データの重要な前処理の選択にどのように役立ったかをまとめてみましょう。

- **Missing Values:** データには欠損値がありません。しかし、新しい未見のデータに欠損値がある場合に備えて、パイプラインにはインプータが必要です（今回は該当しません）。`setup()`関数を実行すると，インピュータが自動的に作成され，パイプラインに保存されます．デフォルトでは，数値に対しては平均インプータを，カテゴリに対しては定数インプータを使用します。これは、`setup()`の`numeric_imputation`と`categorical_imputation`のパラメータを使って変更できます。  
- **Combine Rare Levels:** データセット内の `Clarity` フィーチャーの分布に注目してください。7つの異なるクラスがありますが、そのうち `FL` は4回しか現れません。同様に、`Cut`特徴においても、`Fair`レベルはトレーニングデータセットの中で`2.1%`の頻度でしか現れません。ここでは、設定にある「combine_rare_categories」パラメータを使って、レアなレベルを組み合わせることにします。
- **データのスケール／範囲：** 「Carat Weight」のスケール／範囲が「Price」の変数と大きく異なることに注目してください。Carat Weightは0.75から2.91の範囲であるのに対し、Priceは2,184から101,561までの範囲となっています。
- **Target Transformation:** ターゲット変数 `Price` は正規分布ではありません。尖り度が高く、右に傾いています。ターゲット変数に線形変換を適用するために、セットアップの`transform_target`パラメータを使用します。
- **数値特徴の抽出：** `Carat Weight` は唯一の数値特徴です。そのヒストグラムを見ると、分布は自然な切れ目があるように見えます。ビン化することで、カテゴライズされた特徴に変換し、`sturges' ruleを用いていくつかのレベルを作成します。これにより、線形アルゴリズムのノイズを取り除くことができます。 

In [2]:
#check the shape of data
dataset.shape

(6000, 8)

見たことのないデータに対する`predict_model()`関数のデモを行うために、元のデータセットから600個のサンプルを取り除いて予測に使用しています。これは訓練/テストの分割と混同してはいけません。この分割は実際のシナリオをシミュレートするために行われます。別の見方をすれば、機械学習の実験が行われた時点では、この600件のレコードは利用できなかったということになります。

In [3]:
data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions ' + str(data_unseen.shape))

Data for Modeling: (5400, 8)
Unseen Data For Predictions (600, 8)


# 5.0 Setting up Environment in PyCaret

前回のチュートリアル __[Regression (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ では、`setup()`を使ってpycaretの環境を初期化する方法を学びました。前回の例では、前処理を行わなかったため、追加のパラメータは渡されませんでした（PyCaretによって自動的に実行された機械学習実験に必須のものを除く）。この例では、`setup()`を使って前処理パイプラインをカスタマイズすることで、次のレベルに進みます。前述のセクション4で説明したすべてのステップを実装する方法を見てみましょう。

In [4]:
from pycaret.regression import *

In [5]:
exp_reg102 = setup(data = data, target = 'Price', session_id=123,
                  normalize = True, transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95, 
                  bin_numeric_features = ['Carat Weight'],
                  log_experiment = True, experiment_name = 'diamond1') 

,Description,Value
0,session_id,123
1,Target,Price
2,Original Data,"(5400, 8)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3779, 39)"


これは __[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ で示したのと同じセットアップ・グリッドであることに注意してください。ここでの唯一の違いは、`setup()`に渡されたカスタマイズ・パラメータが`True`に設定されていることです。また、`session_id` が初級チュートリアルで使用したものと同じであることにも注目してください。これは、ランダム化の効果が完全に分離されていることを意味します。この実験で得られた改善点は、`setup()`で行われた前処理や、このチュートリアルの後のセクションで使用されたその他のモデリング技術によるものです。

# 6.0 Comparing All Models

Regression Tutorial (REG101) - [Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__と同様に、このチュートリアルでも`compare_models()`で始めます。そして、以下の結果を前回の実験と比較します。

In [ ]:
top3 = compare_models(exclude = ['ransac'], n_select = 3)

"compare_models"の中で "n_select"パラメータを使用していることに注目してください。前回のチュートリアルでは、compare_models はデフォルトで最もパフォーマンスの高いモデル（デフォルトのソート順に基づいた単一のモデル）を返すことをご紹介しました。しかし、`n_select` パラメータを使えば、上位N個のモデルを返すことができます。この例では、`compare_models` は上位3つのモデルを返しています。

In [ ]:
type(top3)

In [ ]:
print(top3)

比較のために，「RMSLE」スコアを使用します．setup()`でいくつかの前処理を行った後、いくつかのアルゴリズムがどれほど劇的に改善されたかに注目してください。
 
- 線形回帰 RMSLE が `0.6690` から `0.0973` に改善
- リッジ回帰 RMSLE が `0.6689` から `0.0971` に改善
- Huber 回帰 RMSLE が `0.4333` から `0.0972` に改善

前のチュートリアルのすべてのモデルの結果を見るには、__[回帰チュートリアル(REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__のセクション7を参照してください。

# 7.0 Create a Model

前回のチュートリアル __[Regression (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ では、`create_model()`関数を使ってモデルを作成する方法を学びました。ここでは、便利な他のいくつかのパラメータについて学びます。このセクションでは、5フォールドのクロスバリデーションを使用してすべてのモデルを作成します。これを実現するために、`create_model()`の中で`fold`パラメータを渡していることに注目してください。

# 7.1 Create Model (with 5 Fold CV)

In [ ]:
dt = create_model('dt', fold = 5)

# 7.2 Create Model (Metrics rounded to 2 decimals points)

In [ ]:
rf = create_model('rf', round = 2)

`create_model()`の中で`round`パラメータを渡すと、評価指標が10進数2桁に丸められていることに注目してください。

# 7.3 Create Model (KNN)

In [ ]:
knn = create_model('knn')

In [ ]:
print(knn)

# 8.0 Tune a Model

前回のチュートリアル __[Regression (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__ では、あらかじめ定義されたグリッドを使って、モデルのハイパーパラメータを自動的にチューニングする方法を学びました。ここでは、`tune_model()`の`n_iter`というパラメータを紹介します。`n_iter` は，ランダムグリッド探索の反復回数です．反復ごとに，モデルはあらかじめ定義されたハイパーパラメータのグリッドから1つの値をランダムに選択します．デフォルトでは，このパラメータは `10` に設定されています．これは，ハイパーパラメータの最適な値を見つけるために，最大10回の反復を行うことを意味します．この値を大きくするとパフォーマンスが向上しますが，同時にトレーニング時間も長くなります．以下の例をご覧ください。

In [ ]:
tuned_knn = tune_model(knn)

In [ ]:
tuned_knn2 = tune_model(knn, n_iter = 50)

`n_iter`パラメータに基づいて2つのチューニングされたK個の近傍探索が作成されたことに注目してください。`tuned_knn`では、`n_iter`パラメータはデフォルト値のままで、R2は`0.6504`となりました。`tuned_knn2`では，`n_iter`パラメータを`50`に設定したところ，R2は`0.6689`に改善しました．以下に，`tuned_knn`と`tuned_knn2`のハイパーパラメータの違いを見てみましょう．

In [ ]:
plot_model(tuned_knn, plot = 'parameter')

In [ ]:
plot_model(tuned_knn2, plot = 'parameter')

# 9.0 Ensemble a Model

アンサンブルは、モデル（主にツリーベース）のパフォーマンスを向上させるために使用される一般的な機械学習技術です。アンサンブルには様々な手法がありますが、ここではBaggingとBoostingについて説明します。その中にはBaggingやBoosting __[(Read More)](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205)__があります。ここでは、PyCaretの`ensemble_model()`関数を使用します。この関数は、`method`パラメータで定義された手法を用いて、学習された基本推定量をアンサンブルします。

In [ ]:
# lets create a simple dt
dt = create_model('dt')

# 9.1 Bagging

In [ ]:
bagged_dt = ensemble_model(dt)

In [ ]:
# check the parameter of bagged_dt
print(bagged_dt)

アンサンブルによって，RMSLE が `0.1082` から `0.0938` に改善されたことに注目してください．上の例では，`ensemble_model()` のデフォルトのパラメータを使用し，`Bagging` メソッドを使用しました．それでは、`ensemble_model()`の`method`パラメータを変更して`Boosting`を試してみましょう。以下の例をご覧ください。

# 9.2 Boosting

In [ ]:
boosted_dt = ensemble_model(dt, method = 'Boosting')

PyCaretでは、モデルのアンサンブルがとても簡単にできることに注目してください。`method`パラメータを変更するだけで、何行ものコードを必要とするBaggingやBoostingを行うことができます。なお、`ensemble_model()` はデフォルトで `10` 個の推定量を構築します。これは `n_estimators` パラメータで変更することができます。推定量の数を増やすことで，結果が改善されることがあります．以下の例をご覧ください。

In [ ]:
bagged_dt2 = ensemble_model(dt, n_estimators=50)

n_estimators パラメータを増やすことで、どのように結果が改善されるかに注目してください。デフォルトの `10` 推定量の bagged_dt モデルでは、RMSLE は `0.0996` でしたが、`n_estimators = 50` の bagged_dt2 では、RMSLE は `0.0911` に改善されました。

# 9.3 Blending

ブレンディングは、PyCaretで使用できるアンサンブルのもう一つの一般的な手法です。複数のモデルを作成し、個々の予測値を平均化して最終的な予測値を形成します。以下に例を挙げてみましょう。

In [ ]:
# train individual models to blend
lightgbm = create_model('lightgbm', verbose = False)
dt = create_model('dt', verbose = False)
lr = create_model('lr', verbose = False)

In [ ]:
# blend individual models
blender = blend_models(estimator_list = [lightgbm, dt, lr])

In [ ]:
# blend top3 models from compare_models
blender_top3 = blend_models(top3)

In [ ]:
print(blender_top3.estimators_)

これで `blend_models()` 関数を使って `VotingRegressor` を作成することができました。`blend_models` 関数が返すモデルは、`create_model()` や `tune_model()` を使って作成した他のモデルと同じです。このモデルは，他のモデルと同じように，`predict_model()`を使って，見たことのないデータを予測するのに使うことができます．

# 9.4 Stacking

アンサンブルの手法としては、スタッキングもよく知られていますが、実用上の難しさから実装されることはあまりありません。スタッキングは、複数のモデルをメタモデルを介して結合するアンサンブル学習の手法です。スタッキングを別の方法で考えると，結果を予測するために複数のモデルが学習され，それらのモデルからの予測値を元の特徴量と一緒に入力として使用するメタモデルが作成されます．`stack_models()`の実装は、Wolpert, D. H. (1992b).Stacked generalization __[(Read More)](https://www.sciencedirect.com/science/article/abs/pii/S0893608005800231)__。

以下、`compare_models`で得られた上位3つのモデルを使った例を見てみましょう。

In [ ]:
stacker = stack_models(top3)

デフォルトでは、メタモデル（予測を生成する最終的なモデル）はLinear Regressionです。メタモデルは、`meta_model`パラメータを使って変更できます。以下に例を示します。

In [ ]:
xgboost = create_model('xgboost')
stacker2 = stack_models(top3, meta_model = xgboost)

このセクションを終える前に、`stack_models()`には、`restack`というまだ見たことのないパラメータがあります。このパラメータは、生データをメタモデルに公開するかどうかを制御します。`True`に設定すると、ベースレベルモデルのすべての予測値とともに、生データをメタモデルに公開します。デフォルトでは、`True`に設定されています。以下の例では、`restack`パラメータを`False`に変更しています。

# 10.0 Experiment Logging

PyCaret 2.0は、機械学習コードの実行時にパラメータ、コードバージョン、メトリクス、出力ファイルをログに記録し、後で結果を可視化するためのバックエンドAPIとUIとして、MLflow Trackingコンポーネントを組み込んでいます。pycaretで実験のログを取るには、この例のようにsetup関数でlog_experimentとexperiment_nameパラメータを使うだけです。

UIは、`localhost:5000`で起動できます。コマンドラインやノートブックからMLFlowサーバーを起動するだけです。以下の例をご覧ください。

In [ ]:
# to start the MLFlow server from notebook:
!mlflow ui

# Open localhost:5000 on your browser (below is example of how UI looks like)
![title](https://i2.wp.com/pycaret.org/wp-content/uploads/2020/07/classification_mlflow_ui.png?resize=1080%2C508&ssl=1)

# 11.0 Wrap-up / Next Steps?

このチュートリアルでは、多くの新しいコンセプトを取り上げました。最も重要なことは、`setup()`でパイプラインをカスタマイズするために探索的データ解析を使用する方法を見たことです。これにより、以前に__[Regression Tutorial (REG101) - Level Beginner](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb)__で見たものと比較して、結果がかなり改善されました。また、PyCaretでのアンサンブルの実行とチューニングの方法も学びました。

しかし、`pycaret.regression`には、shapley値を用いたより複雑な木ベースのモデルの解釈や、多層スタックネットなどの高度なアンサンブル技術、より多くの前処理パイプラインメソッドなど、さらに高度な内容があります。これらの内容については、次回、`pycaret.regression`シリーズの最終回のチュートリアルでご紹介します。

それでは、次回のチュートリアルでお会いしましょう。_[回帰チュートリアル(REG103) - Level Expert](https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Expert%20-%20REG103.ipynb)__へのリンクをたどってください。